[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gsarti/ik-nlp-tutorials/blob/main/notebooks/W2E_Pipelines_Sentence_Transformers.ipynb)

In [1]:
# Run in Colab to install local packages
!pip install transformers sentencepiece torch datasets sentence-transformers

# 🤗 Pipelines & Sentence Transformers for Semantic Search and QA

*This notebook is partly adapted from a tutorial by Wietse De Vries for the IK-NLP course of 2021*

The goal of this notebook is to make you practice with some pipeline use-cases and to introduce the [Sentence Transformers](https://sbert.net/) in the context of a concrete example of semantic search and question answering (relevant to the Open-book Question Answering final project).

Exercises 1 and 2 of this notebook are mandatory and will be part of your first graded portfolio. Exercise 3 is optional, but we highly recommend you to complete it, especially if you're interested in the Open-book Question Answering final project.

## Exercise 1: Using the Fill-mask pipeline for Probing Linguistic Knowledge in mBERT

As you probably know by now, BERT is a transformed-based, context-sensitive, neural language models that has been trained, among others, on a masked language modeling task, where the model learns to predict what the most likely word is at a a *masked* (hidden) position in the sentence. In a sentence like:

>There were several [MASK] with the proposed solution.
    
the model will learn that the word *problems* or *issues* is more likely at this position than the word *unicorns* or *days*. The model uses both the right and left context of the masked position to make its predictions. Models trained to maximize the probability of predicting the correct masked words learn to represent a good amount of linguistic knowledge as a result of this.

A **probe** is a test of a language model aimed at investigating how accurate these predictions are, especially for cases where syntax makes it quite clear that one (form of a) word is correct, and another word is impossible. In the example above, for instance, the masked position can be filled by a plural noun (*problems*) but not by a singular noun (*problem*). If the model makes predictions that respect the linguistic constraints, we have reason to believe that the model is somehow aware of the linguistic structure of the language.

While predicting whether the masked position should be filled by a singular or plural noun seems easy in the example above (both *were* and *several* are good predictors of plural), we can try to make the task harder by looking for contexts where the solution requires more careful *attention* to the right words in the context

>There were some [MASK] with the proposed solution.
>
>There could be several [MASK] with the proposed solution.
>
>There were some unexpected and unforeseen [MASK] with the proposed solution.
    
In the examples above, the task is made harder by replacing *several* (which is always followed by a plural noun) by *some* (which can be followed by a singular or a plural noun), by replacing *were* (which always heads a sentence with a plural subject) by *could be* (which can head a sentence with a singular or plural subject), and by inserting material between the verb *were* (which indicates that there should be a plural) and the MASK.


### Assignment

Think of a grammatical phenomenon in a language of your choice, and come up with at least 5 example sentences to probe whether the model makes the correct predictions. Think of cases where the context makes it clear that the mask has to be plural or singular, that a verb has to have a particular form (like plural or singular, or participle or infinitive), that a specific (personal, possessive, reflexive) pronoun has to be used, that an adjective or noun has to have a specific inflection (like in German and more generally in languages with a rich case and/or gender marking system). There is a host of literature on this, see for instance [Marvin and Linzen](https://arxiv.org/abs/1808.09031) (for English) and [Sahin et al](https://www.mitpressjournals.org/doi/full/10.1162/coli_a_00376) (for multilingual probes).

### Model

The model we will be using for this task is the [multilingual BERT](https://huggingface.co/bert-base-multilingual-cased) model mBERT, that was trained on the Wikipedia text of the 102 largest Wikipedia's. This means that you do not have to choose examples from English, but that you may also present a probe for another language.

The following loads the pipeline for doing masked prediction, and load the mBERT model (this may take a minute or so). You can ignore the warning about some weights not being initialized. The pipeline can be used to test masked language model prediction. Given a sequence containing the special token [MASK], the model will predict what the most likely tokens are at that position, using both left and right context.

In [2]:
from transformers import pipeline

mbert = pipeline('fill-mask', model='bert-base-multilingual-cased')
mbert('There were some unexpected and unforeseen [MASK] with the proposed solution.')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/202 [00:00<?, ?it/s]

BertForMaskedLM LOAD REPORT from: bert-base-multilingual-cased
Key                         | Status     |  | 
----------------------------+------------+--+-
bert.pooler.dense.bias      | UNEXPECTED |  | 
cls.seq_relationship.bias   | UNEXPECTED |  | 
cls.seq_relationship.weight | UNEXPECTED |  | 
bert.pooler.dense.weight    | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

[{'score': 0.5272426605224609,
  'token': 20390,
  'token_str': 'problems',
  'sequence': 'There were some unexpected and unforeseen problems with the proposed solution.'},
 {'score': 0.09590253233909607,
  'token': 17850,
  'token_str': 'issues',
  'sequence': 'There were some unexpected and unforeseen issues with the proposed solution.'},
 {'score': 0.06644545495510101,
  'token': 64557,
  'token_str': 'difficulties',
  'sequence': 'There were some unexpected and unforeseen difficulties with the proposed solution.'},
 {'score': 0.016898171976208687,
  'token': 73082,
  'token_str': 'dealing',
  'sequence': 'There were some unexpected and unforeseen dealing with the proposed solution.'},
 {'score': 0.015166707336902618,
  'token': 18107,
  'token_str': 'associated',
  'sequence': 'There were some unexpected and unforeseen associated with the proposed solution.'}]

By default, the pipe returns the 5 most likely words that could appear at the position of the mask, along with a score. If you want to know specifically whether the model prefers one of two forms, you can give these forms as targets to the pipe, and also print the answer in a more readable form:

In [5]:
def probe(sentence: str, targets: str) :
    for res in mbert(sentence,targets=targets) :
        print(f"{res['score']:6.4f}\ttoken: {res['token_str']}\t{res['sequence']}")

probe('There were some unexpected and unforeseen [MASK] with the proposed solution.',['challenge','challenges'])

0.0034	token: challenges	There were some unexpected and unforeseen challenges with the proposed solution.
0.0001	token: challenge	There were some unexpected and unforeseen challenge with the proposed solution.


> **💡 Interesting Fact**: The same bias that is present towards grammatically correct choices can be observed in other cases, such as racial and gender stereotyping. Much work is currently in process to identify and remove gender and racial biases from learned language embeddings. See the following example and [this recent survey on the topic](https://arxiv.org/abs/2112.14168).

In [6]:
from transformers import pipeline

unmasker = pipeline("fill-mask", model="bert-base-cased")
result = unmasker("This man works as a [MASK].")
print([r["token_str"] for r in result])

result = unmasker("This woman works as a [MASK].")
print([r["token_str"] for r in result])

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/202 [00:00<?, ?it/s]

BertForMaskedLM LOAD REPORT from: bert-base-cased
Key                         | Status     |  | 
----------------------------+------------+--+-
bert.pooler.dense.bias      | UNEXPECTED |  | 
cls.seq_relationship.bias   | UNEXPECTED |  | 
cls.seq_relationship.weight | UNEXPECTED |  | 
bert.pooler.dense.weight    | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

['lawyer', 'carpenter', 'doctor', 'waiter', 'mechanic']
['nurse', 'waitress', 'teacher', 'maid', 'prostitute']


### Your Turn to Probe

Give at least five example sentences with a [MASK] and a list of targets that illustrate a specific grammatical phenomenon in a language of your choice. Describe what the grammatical phenomenon is you are investigating. Use the probe function for testing. Try to include both *easy* sentences (where the model should do well) as well as *hard* sentences (where there are words in the context that might lead to confusion, or where the clue words are far away from the mask). For languages other than Dutch or English, make sure to include enough explanation so that examples and tests are clear to a non-native speaker.

Describe how well the model did on your probe sentences. Where there any cases where the model made the wrong decision?

**Grammatical phenomenon:** Subject–adjective agreement in Italian. In Italian, adjectives must agree in gender (masculine/feminine) and number (singular/plural) with the noun they modify.

**Objective:** Test whether the language model can correctly predict the form of an adjective given the subject, including cases where the correct agreement might be obscured by distractor words or complex sentence structures.

**Design of the probe sentences:**

I created five sentences with a [MASK] for the adjective. Some sentences are easy, where the subject and adjective are close and unambiguous. Some sentences are hard, where there are distractor nouns, negations, or compound subjects that may confuse the model. For each sentence, I provide a list of possible targets.

**Examples:**

“La casa è [MASK].” → target: *grande* (fem. sing.) → tr. "The house is [big]."

“I fiori sono [MASK].” → target: *colorati* (masc. pl.) → tr. "The flowers are [colorful]."

“Nonostante le nuvole scure, il cielo oggi sembra [MASK].” → target: *sereno* (masc. sing.) → tr. "Despite the dark clouds, the sky looks [clear] today."

“Le ragazze del coro erano tutte [MASK].” → target: *eleganti* (fem. pl.) → tr. "The choir girls were all [elegant]."

“Il ragazzo e la ragazza sono [MASK].” → target: *felici* (pl., masc+fem) → tr. "The boy and the girl are [happy]."

In [18]:
# Loading an Italian BERT model for masked language modeling
from transformers import pipeline

italian_model = pipeline("fill-mask", model="dbmdz/bert-base-italian-cased")

Loading weights:   0%|          | 0/202 [00:00<?, ?it/s]

BertForMaskedLM LOAD REPORT from: dbmdz/bert-base-italian-cased
Key                         | Status     |  | 
----------------------------+------------+--+-
bert.pooler.dense.bias      | UNEXPECTED |  | 
cls.seq_relationship.bias   | UNEXPECTED |  | 
cls.seq_relationship.weight | UNEXPECTED |  | 
bert.pooler.dense.weight    | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [19]:
# Probe function
def probe(sentence: str, targets: list):
    for res in italian_model(sentence, targets=targets):
        print(f"{res['score']:6.4f}\ttoken: {res['token_str']}\t{res['sequence']}")

# Probe sentences
probe('La casa è [MASK].', ['grande', 'grandi'])
probe('I fiori sono [MASK].', ['colorato', 'colorata', 'colorati', 'colorate'])
probe('Nonostante le nuvole scure, il cielo oggi sembra [MASK].', ['sereno', 'serena', 'sereni', 'serene'])
probe('Le ragazze del coro erano tutte [MASK].', ['elegante', 'eleganti'])
probe('Il ragazzo e la ragazza sono [MASK].', ['felice', 'felici'])

The specified target token `colorata` does not exist in the model vocabulary. Replacing with `colora`.
The specified target token `sereno` does not exist in the model vocabulary. Replacing with `sere`.
The specified target token `serene` does not exist in the model vocabulary. Replacing with `sere`.


0.0090	token: grande	La casa è grande.
0.0000	token: grandi	La casa è grandi.
0.0011	token: colorati	I fiori sono colorati.
0.0000	token: colorate	I fiori sono colorate.
0.0000	token: colorato	I fiori sono colorato.
0.0000	token: colora	I fiori sono colora.
0.0008	token: serena	Nonostante le nuvole scure, il cielo oggi sembra serena.
0.0000	token: sere	Nonostante le nuvole scure, il cielo oggi sembra sere.
0.0000	token: sereni	Nonostante le nuvole scure, il cielo oggi sembra sereni.
0.0020	token: eleganti	Le ragazze del coro erano tutte eleganti.
0.0000	token: elegante	Le ragazze del coro erano tutte elegante.
0.0455	token: felici	Il ragazzo e la ragazza sono felici.
0.0007	token: felice	Il ragazzo e la ragazza sono felice.


The Italian BERT model correctly predicted adjective agreement in most cases. It performed well on simple sentences where the subject and adjective were close (“La casa è [MASK]”) and also handled plural agreement correctly (“I fiori sono [MASK]”).

The model also correctly applied the masculine plural rule in compound subjects (“Il ragazzo e la ragazza sono [MASK]”), showing that it has learned agreement rules beyond simple proximity.

However, the model failed in the sentence with a distractor (“Nonostante le nuvole scure, il cielo oggi sembra [MASK]”). It incorrectly selected the feminine form “serena” instead of the correct masculine “sereno”, maybe because the nearby noun “nuvole” is feminine. This suggests that the model may rely on local cues rather than full syntactic structure in more complex sentences.

Overall, the model captures basic agreement patterns but can be misled by intervening nouns.

## Exercise 2: Mixing Pipelines for Text-to-text QA in Many Languages

The Model Hub of HuggingFace is home to a staggering amount of models for the more disparate use-cases, but you may have noticed that many of those are trained on the English language. Let's consider for example the [`UnifiedQA`](https://github.com/allenai/unifiedqa) model by AllenAI, which is a T5 model architecture trained to perform question answering on multiple formats (e.g. extract the answer from the provided context, produce an answer without a supporting context, choose among multiple possible answers, yes/no questions) using a unified text-to-text approach. While this opens thrilling perspectives in having a single model for all QA use-cases, UnifiedQA models are available for English only, and training such models from scratch in another language would require nontrivial effort and resources.

Here are several examples of how text should be formatted for the UnifiedQA model:

| **Task type** | **Example Dataset** | **Format** | **Example** | **Output** |
| :---: | :--- | :--- | :--- | :--- |
|  **Extractive QA** | SQUAD | `<QUESTION> \n <CONTEXT>` | `At what speed did the turbine operate? \n (Nikola_Tesla) On his 50th birthday in 1906, Tesla demonstrated his 200 horsepower (150 kilowatts) 16,000 rpm bladeless turbine. ...` |  `16,000 rpm` |
|  **Abstractive QA** | NarrativeQA | `<QUESTION> \n <CONTEXT>` | `What does a drink from narcissus's spring cause the drinker to do?  \n  Mercury has awakened Echo, who weeps for Narcissus, and states that a drink from Narcissus's spring causes the drinkers to ''Grow dotingly enamored of themselves.'' ...` | `fall in love with themselves` |
|  **Multiple-choice QA** | ARC-challenge | `<QUESTION> \n (a) <CHOICE_A> (b) <CHOICE_B> ...` | `What does photosynthesis produce that helps plants grow? \n (A) water (B) oxygen (C) protein (D) sugar` | `sugar` |
|  **Multiple-choice QA with context** | MCTest | `<QUESTION> \n (a) <CHOICE_A> (b) <CHOICE_B> ... \n <CONTEXT>` | `Who was Billy? \n (A) The skinny kid (B) A teacher (C) A little kid (D) The big kid \n Billy was like a king on the school yard. A king without a queen. He was the biggest kid in our grade, so he made all the rules during recess. ...` | `The big kid` |
|  **Yes-no QA** | BoolQ | `<QUESTION> \n <CONTEXT>` | `Was America the first country to have a president?  \n (President) The first usage of the word president to denote the highest official in a government was during the Commonwealth of England ...` | `no` |

### Assignment

We are gonna build a function making use of multiple models through 🤗 Pipelines to generate a response to a question in one of the formats specified above, in one of the languages supported by the MT systems available on the HuggingFace Model Hub. The function will translate and paraphrase the query into multiple examples, and then pick the best outputs of the UnifiedQA model as candidates for backtranslation. In this way, we mock the existance of a UnifiedQA model for the language of our choice.

### Model

The following code loads the UnifiedQA model and use it to perform QA on a multiple choice question without context.

In [ ]:
from transformers import pipeline

# Using at least the base variant of the model is advised for good results
generator = pipeline("text2text-generation", model="allenai/unifiedqa-t5-base")
generator("What is the name of the city where the Eiffel Tower is located? \n (A) Paris (B) London (C) Prague (D) Berlin")

[{'generated_text': 'Paris'}]

### Your turn to Pipe

Using the pipelines we saw in the tutorial, create a function taking a `question` string, an optional `context` string and an optional list of strings called `choices` and performs the following steps:

- Use one of the [MarianMT](https://huggingface.co/models?search=helsinki-nlp) machine translation models to translate all the inputs from the language of your choice to English. You may want to split the text into sentences (e.g. by splitting on periods) to obtain better results for the context.

- Use a paraphrasing model ([`tuner007/pegasus_paraphrase`](https://huggingface.co/tuner007/pegasus_paraphrase) is a good choice, albeit heavy) to produce 4 paraphrases of the question, using the `num_return_sequences` parameter.

- For each question (translated + 4 paraphrases), format it with the translated context and choices (if present) as a single string in the format required by UnifiedQA.

- Use the `allenai/unifiedqa-t5-base` model to generate an answer for each of the 5 questions.

- If at least 3 of the 5 answers are identical strings, return the result translated back to the original language using the MarianMT model for the reciprocal language pair (e.g. if you used `Helsinki-NLP/opus-mt-nl-en` to translate from Dutch to English, you will need to use `Helsinki-NLP/opus-mt-en-nl`). Otherwise, print "No common answer found" translated in the original language.

**Importantly**, the quality of the output does not determine your score in the evaluation. The goal is to get a feel for the models and their capabilities.

In [ ]:
from typing import Optional, List

def answer(question: str, context: Optional[str] = None, choices: Optional[List[str]] = None):
    # Your beautiful function

## (Optional) Exercise 3: SentenceTransformers for Semantic Similarity Search

*Figures and some code are from the [CohereAI Semantic Search Tutorial](https://docs.cohere.ai/semantic-search/) by Jay Alammar.*

> SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. The initial work is described in our paper [Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks](https://arxiv.org/abs/1908.10084).
>
> You can use this framework to compute sentence / text embeddings for more than 100 languages. These embeddings can then be compared e.g. with cosine-similarity to find sentences with a similar meaning. This can be useful for semantic textual similar, semantic search, or paraphrase mining.
>
> The framework is based on PyTorch and Transformers and offers a large collection of pre-trained models tuned for various tasks. Further, it is easy to fine-tune your own models.

Semantic search is a typical use case in natural language processing, in which we want to retrieve the most relevant documents from a corpus (e.g. Automatic FAQs, Web browser search results, etc.). Using the similarity between embedded text representations allows us to go beyond simple keyword matching, which is highly desirable in this setting (e.g. `tomorrow will rain` should be very close in embedding space to `the weather forecast announces showers for the next day`, despite no lexical overlap).

<div>
<img alt="Visualizing Semantic Search" src="https://github.com/cohere-ai/notebooks/raw/main/notebooks/images/basic-semantic-search-overview.png?3" style="width: 60%" />
</div>

In this exercise we will use first Huggingface Transformers and then the [SentenceTransformers](https://sbert.net) library to find the most relevant paragraphs for a specific query.

Let's start by loading the `train` split of the `squad` dataset from the Dataset Hub and flatten its structure so that every example contains a single triplet `(context, question, answer)`. We are going to use only the first 50 rows of the dataset, the others can be discarded.

In [ ]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:50]")

squad_train_filtered = None# Your code here

Now, sample a question at random from the resulting dataset and print it. You can use `shuffle` or turn the Dataset into a `DataFrame` and use `sample`.

In [ ]:
query = None # Your code here

print(query)

We now going to use a model trained to perform **semantic search** to retrieve the top 10 most likely contexts for each selected question.

The model `sentence-transformers/multi-qa-MiniLM-L6-cos-v1` is a good choice for this task, since it is relatively small and was explicitly trained for semantic search. We are going to define now three utility functions:

- `dot_score` computes the dot product between two Pytorch tensors.
- `mean_pooling` averages the embeddings produced by a model to obtain a **sentence embedding**.
- `encode` uses a model and a tokenizer to convert a list of texts into a tensor of embeddings. **Complete it with the first two steps we saw in the tutorial.**

In [ ]:
import torch
from torch import Tensor
import torch.nn.functional as F


def dot_score(a: Tensor, b: Tensor):
    """
    Computes the dot-product dot_prod(a[i], b[j]) for all i and j.
    :return: Matrix with res[i][j]  = dot_prod(a[i], b[j])
    Taken from the SentenceTransformer library
    """
    if not isinstance(a, torch.Tensor):
        a = torch.tensor(a)
    if not isinstance(b, torch.Tensor):
        b = torch.tensor(b)
    if len(a.shape) == 1:
        a = a.unsqueeze(0)
    if len(b.shape) == 1:
        b = b.unsqueeze(0)
    print(a.shape, b.shape)
    # Compute the dot-product
    return torch.mm(a, b.transpose(0, 1))


#Mean Pooling - Average all the embeddings produced by the model
def mean_pooling(model_output, attention_mask):
    # First element of model_output contains all token embeddings
    token_embeddings = model_output.last_hidden_state
    # Expand the mask to the same size as the token embeddings to avoid indexing errors
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    # Compute the mean of the token embeddings
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


#Encode text
def encode(model, tokenizer, texts):
    # Tokenize sentences
    encoded_input = None # Your code here
    # Compute token embeddings
    with torch.no_grad():
        model_output = None # Your code here
    # Perform pooling
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)

    return embeddings

Let's now use these functions to compute similarity scores for the query and all the contexts:

In [ ]:
# Sentences we want sentence embeddings for
contexts = list(squad_train_filtered.to_pandas()['context'])

# Load the model and tokenizer from HuggingFace Hub
tokenizer = None # Your code here
model = None # Your code here

#Encode query and contexts with the encode function
query_emb = None # Your code here
contexts_emb = None # Your code here

#Compute dot score between query and all contexts embeddings
scores = torch.mm(query_emb, contexts_emb.transpose(0, 1))[0].cpu().tolist()

#Combine contexts & scores
contexts_score_pairs = list(zip(contexts, scores))

#Sort by decreasing score
contexts_score_pairs = sorted(contexts_score_pairs, key=lambda x: x[1], reverse=True)

#Output passages & scores
for ctx, score in contexts_score_pairs:
    print(score, ctx)

We can now use SentenceTransformers to do the same, but using much less code:

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Query was defined above
contexts = list(squad_train_filtered.to_pandas()['context'])

#Load the model
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

#Encode query and contexts using SentenceTransformer model.encode
query_emb = model.encode(query)
contexts_emb = model.encode(contexts)

#Compute dot score between query and all contexts embeddings
scores = util.dot_score(query_emb, contexts_emb)[0].tolist()

#Combine contexts & scores
contexts_score_pairs = list(zip(contexts, scores))

#Sort by decreasing score
contexts_score_pairs = sorted(contexts_score_pairs, key=lambda x: x[1], reverse=True)

#Output passages & scores
for ctx, score in contexts_score_pairs:
    print(score, ctx)

For a more advanced overview on how to optimize speed with semantic search, see how to use the [FAISS](https://github.com/facebookresearch/faiss) library to perform fast nearest neighbor search natively with Huggingface Transformers here: [Using FAISS for efficient similarity search ](https://huggingface.co/course/chapter5/6?fw=pt#using-faiss-for-efficient-similarity-search)